In [ ]:
# Checks that `o` is an instance of `t` (ex: integer, list).
# Produces a clear error message otherwise.
# This function is not essential but can help a lot for debugging.
def check_type(o, t, name=None):
    if(name is None): name = "[no name]"
    assert isinstance(o, t), (f"Type problem: variable {name} (type: {type(o)}; value: {o}) is not an instance of {t}")

In [ ]:
# Example 1:
check_type([1,2,3], list) # Works fine because [1,2,3] is indeed a list.

In [ ]:
# Example 2:
# check_type(1, list) # An AssertionError exception is raised because 1 is not a list.

In [ ]:
# Constant
class C:
    # name: string
    def __init__(self, name):
        check_type(name, str, "name")

        self._name = name

    # Defines the behaviour of "==".
    # In this case: two C·s are considered equal if they have the same `_name`.
    def __eq__(self, other):
        return isinstance(other, C) and self._name == other._name

    # Required to be able to use the class in sets or dictionaries.
    def __hash__(self):
        return hash(self._name)

    # Returns a string representation of the object. Used to print the object in a readable way.
    def __str__(self):
        return self._name

    # Returns a string representation of the object. Also used to print the object in a readable way.
    def __repr__(self):
        return str(self)

In [ ]:
# Variable
class V:
    # name: string
    def __init__(self, name):
        check_type(name, str, "name")

        self._name = name

    # Defines the behaviour of "==".
    # In this case: two V·s are considered equal if they have the same `_name`.
    def __eq__(self, other):
        return isinstance(other, V) and self._name == other._name

    # Required to be able to use the class in sets or dictionaries.
    def __hash__(self):
        return hash(self._name)

    # Returns a string representation of the object. Used to print the object in a readable way.
    def __str__(self):
        return self._name

    # Returns a string representation of the object. Also used to print the object in a readable way.
    def __repr__(self):
        return str(self)

In [ ]:
# Predicate
class P:
	# name: string
	def __init__(self, name, arity):
		check_type(name, str, "name")
		check_type(arity, int, "arity")

		self._name = name
		self.arity = arity

	# Defines the behaviour of "==".
	# In this case: two P·s are considered equal if they have the same `_name` and the same `arity`.
	def __eq__(self, other):
		return isinstance(other, P) and (self._name == other._name) and (self.arity == other.arity)

	# Required to be able to use the class in sets or dictionaries.
	def __hash__(self):
		return hash((self._name, self.arity))

	# Returns a string representation of the object. Used to print the object in a readable way.
	def __str__(self):
		return self._name

	# Returns a string representation of the object. Also used to print the object in a readable way.
	def __repr__(self):
		return str(self)

# Instructions
*  Instantiate and then print a constant "Sabine", a variable "x" and a binary predicate "eat".
*  Use `isinstance` to check that these objects instantiate the class you think they instantiate.

In [ ]:
Sabine = C("Sabine")
x = V("x")
eat = P("eat", 2)
print(f"Sabine is instance of C ? {isinstance(Sabine, C)}")
print(f"x is instance of V ? {isinstance(x, V)}")
print(f"eat is instance of P ? {isinstance(eat, P)}")

A model of First Order Logic consists of a domain and an interpretation function.
*  The domain is simply a set the element of which are called "individuals". Here, individuals will be integers.
*  The interpretation function sends any constant to an individual and any predicate to a tuple of individuals (see slides).

In [ ]:
class InterpretationFunc:
	# c_dic: dictionary; keys are C·s, values are integers
	# p_dic: dictionary; keys are P·s, values are sets of tuples of integers
	def __init__(self, c_dic, p_dic):
		self._c_dic = c_dic
		self._p_dic = p_dic

	# Remark: __getitem__ can be called using the []-notation: "i[x]" is translated as "i.__getitem__(x)". Use the []-notation instead of calling __getitem__ explicitly.
	# Returns the interpretation of `x`.
	# x: either a C or a P
	def __getitem__(self, x):
		if(isinstance(x, C)): return self._c_dic[x] # Raises an exception if the constant has no entry in `_c_dic`.
		if(isinstance(x, P)): return self._p_dic.get(x, set()) # Returns an empty set if the predicate has no entry in `_p_dic`.
		raise TypeError

	# Returns the list obtained from `l` by replacing all constants by their interpretation (other elements should appear unaffected).
	# The original list `l` should not be affected.
	# (Be aware that this function returns a list and not a tuple. If you need a tuple, use the `tuple` function to convert the list into one.)
	# l: list of C·s and V·s
	def map(self, l):
		check_type(l, list, "l")
		# new_l: list of C·s and V·s
		new_l = l.copy()
		for i in range(len(l)): # iterate over C·s and V·s or individuals in the domain in l
			if isinstance(l[i], C): # if the current element in the loop in a constant
				new_l[i] = self[l[i]] # assign to the i-th element in new_l the interpretation of the i-th element of l
		return new_l

	# Returns a string representation of the object. Used to print the object in a readable way.
	def __str__(self):
		tmp = list(self._c_dic.items())
		tmp.extend(self._p_dic.items())
		s = ', '.join([f"{k}: {v}" for (k, v) in tmp])
		return '{' + s + '}'

	# Returns a string representation of the object. Also used to print the object in a readable way.
	def __repr__(self):
		return str(self)

# Instructions
*  Complete `InterpretationFunc.map` above.
*  Instantiate an interpretation function `i_func` that interprets the constant "Sabine" as the integer 1, and predicates "tall" "eat" and "like" as non-empty sets. Warning: the one-tuple composed of element `x` is written as `(x,)` in Python (instead of `(x)`, which is just another way to write `x`).
*  Print `i_func`.

In [ ]:
i_func = InterpretationFunc({C("Sabine"): 1}, {P("tall", 1): {(0,)}, P("eat", 2): {(0,)}, P("like", 2): {(0,)}})
print(f"i_func is: {i_func}")

# test InterpretationFunc.map
l = [C("Sabine"), V("x")]
print(f"\noriginal list before mapping: {l}")
print(f"list returned from i_func.map(): {i_func.map(l)}")
print(f"original list after mapping: {l}")

In [ ]:
class Model:
	# domain: set of integers
	# i_func: InterpretationFunc
	def __init__(self, domain, i_func):
		check_type(domain, set, "domain")
		check_type(i_func, InterpretationFunc, "i_func")

		self.domain = domain
		self.i_func = i_func

	# Returns a string representation of the object. Used to print the object in a readable way.
	def __str__(self):
		return f'{{D={self.domain}; I={self.i_func}}}'

	# Returns a string representation of the object. Also used to print the object in a readable way.
	def __repr__(self):
		return str(self)

# Instructions
*  Instantiate a model `model` from a finite domain and the `i_func` interpretation function defined previously.
*  Print `model`.

In [ ]:
model = Model({1,2}, i_func)
print(model)

# Instructions
*  Complete `VarAssignment.assign`.
*  Complete `VarAssignment.map`.

In [ ]:
# For variable assignments.
class VarAssignment:
	# dic: dictionary; keys are Vs, values are integers
	# If `dic` is not specified, the empty dictionary ({}) is used.
	def __init__(self, dic={}):
		check_type(dic, dict, "dic")

		self._dic = dic

	# Returns the variable assignment that only differs from the present one (i.e. `self`) with "x := d". ?
	# The present assignment is not modified and a new assignment is instantiated.
	# x: V
	# d: integer
	def assign(self, x, d):
		check_type(x, V, "x")
		check_type(d, int, "d")
		if (self._dic[x] == d): # if the interpretation of x is the same as d
	 		return self
		else:
			new_dic = self._dic.copy()
			new_dic[x] = d
			return VarAssignment(new_dic)

	# Returns the list obtained from `l` by replacing all variables by their assignments (other elements should appear unaffected).
	# The original list `l` should not be affected.
	# (Be aware that this function returns a list and not a tuple. If you need a tuple, use the `tuple` function to convert the list into one.)
	# l: list
	def map(self, l):
		check_type(l, list, "l")
		new_l = l.copy()
		for i in range(len(l)): # iterate over C·s and V·s or individuals in the domain in l
			if isinstance(l[i], V): # if the current element in the loop in a constant
				new_l[i] = self._dic[l[i]] # assign to the i-th element in new_l the assignment of the i-th element of l
		return new_l

	# Returns a string representation of the object. Used to print the object in a nice way.
	def __str__(self):
		return f'{self._dic}'

In [ ]:
var_assign = VarAssignment({V("x"): 0})

# testing VarAssignment.assign
print(f"Original VarAssignment object called var_assign: {var_assign}")
print(f" -> ")
var_assign.assign(V('x'), 0)
new_var_assign = var_assign.assign(V("x"), 1)
print(f"New var assign object after doing var_assign.assign: {new_var_assign}")
print(f"Original VarAssignment object after doing var_assign.asign: {var_assign}")

# testing VarAssignment.map
print(f"\noriginal list before mapping: {l}")
print(f"list returned from var_assign.map(): {var_assign.map(l)}")
print(f"original list after mapping: {l}")

*  For this TP, a formula is represented by an instance of the class `Formula` (in fact, of some sub-class of `Formula`).
*  There is one sub-class for each "kind" of formulas, that is to say for each clause in the inductive definition of the language of FOL.

In [ ]:
# The general class for logical formulas.
# This class is sub-classed below.
class Formula:
	# Checks whether the formula is true according to the model `m`.
	# The use of this method requires that the formula be closed.
	# This method does almost nothing by itself. All the work is done by the `check` method defined for each kind of formulas (sub-classes of `Formula`).
	# m: Model
	def check_closed(self, m):
		check_type(m, Model, "m")

		f = VarAssignment() # Empty partial variable assignment.
		return self.check(m, f)

# Instructions
*  `PredApp` is the sub-class corresponding to formulas composed of a single predicate with all of its arguments (1st clause in the definition of the language of FOL).
*  Complete `PredApp.check`. (The slide about the valuation of FOL formulas contains all the information you need.)
*  Then, instantiate three formulas (closed or not), using `PredApp` and print  their interpretation according in `model` for some variable assignment (that you have to instantiate).

In [ ]:
# Predicate application
class PredApp(Formula):
	# pred: P
	# args: list of V·s and C·s
	def __init__(self, pred, args):
		check_type(pred, P, "pred")
		assert (pred.arity == len(args)), f"{pred.arity} argument·s expected but {len(args)} given."
		check_type(args, list, "args")

		self._pred = pred
		self._args = args

	# Checks whether the formula is true according to the model `m` and the variable assignment `f`.
	# m: Model
	# f: VarAssignment
	def check(self, m, f):
		check_type(m, Model, "m")
		check_type(f, VarAssignment, "f")
		# after mapping the constants and the variables in the arguments of the formula compare that mapping (tuple) to the interpretation of the predicate (set of tuples)
		return tuple(f.map(m.i_func.map(self._args))) in m.i_func[self._pred]

	# Returns a string representation of the object. Used to print the object in a readable way.
	def __str__(self):
		return f"{self._pred}({','.join([str(x) for x in self._args])})"

In [ ]:
# Defining predicates, constants, variables, and VarAssignment.s
Sabine = C("Sabine")
Jordan = C("Jordan")
Gertrude = C("Gertrude")
hose = C("hose")
tall = P("tall", 1)
like = P("like", 2)
human = P("human", 1)
wooden = P("wooden", 1)
y = V('y')
x = V("x")
f = VarAssignment() # empty variable assignment

new_inter_func = InterpretationFunc({Sabine: 1, Jordan: 0, Gertrude: 2, hose: 3},
 {tall: {(1,)}, like: {(0,2), (0,0)}, human: {(0,), (1,), (2,)}, wooden: {tuple()}})
model = Model({0,1,2,3}, new_inter_func)
print(f"The model is:\n {model} \n")

# Unary Predicates ...
# ... with a constant
pred_sabine_tall = PredApp(tall, [Sabine])
print(f"{pred_sabine_tall}: {pred_sabine_tall.check(model, f)} \n")

# ... with a variable
pred_y_tall = PredApp(tall, [y])
print(f"{pred_y_tall}: {pred_y_tall.check(model, VarAssignment({y: 0}))} (with f(y) = 0) \n")

# ... that apply to multiple individuals in the domain
pred_gertrude_human = PredApp(human, [Gertrude])
print(f"{pred_gertrude_human}: {pred_gertrude_human.check(model, f)} \n")

# Binary predicates ...
# ... with constant
pred_Jordan_like_Sabine = PredApp(like, [Jordan, Sabine])
print(f"{pred_Jordan_like_Sabine}: {pred_Jordan_like_Sabine.check(model, f)} \n")

# ... with variable
pred_Jordan_like_y = PredApp(like, [Jordan, y])
print(f"{pred_Jordan_like_y}: {pred_Jordan_like_y.check(model, VarAssignment({y: 2}))} (with f(y) = 2) \n")

# Instructions
*  Write `Neg`, a sub-class of `Formula`, for negation.
*  This class must possess, in addition to a constructor (`__init__`), a `check` method and a `__str__` one (see `PredApp`; you can use the ¬ symbol in `__str__`).
*  (The slide about the syntax of FOL and the one about its semantics contain all the information you need.)
*  Use `check_type` at the beginning of each method in order to check the type of each argument.
*  Instantiate several formulas using `Neg`, display these formulas and there value in some model for some variable assignment.

In [ ]:
# Negation application
class Neg(Formula):
  # phi: Formula
  def __init__(self, phi):
    check_type(phi, Formula, "phi")
    self._phi = phi

	# Checks whether the formula is true according to the model `m` and the variable assignment `f`.
	# m: Model
	# f: VarAssignment
  def check(self, m, f):
    check_type(m, Model, "m")
    check_type(f, VarAssignment, "f")
    return not(self._phi.check(m, f)) # return the opposite of the value of phi

	# Returns a string representation of the object. Used to print the object in a readable way.
  def __str__(self):
    return f"¬({self._phi})"

In [ ]:
print(f"The model is:\n {model} \n")

# Unary Predicates ...
# ... with a constant
neg_pred_sabine_tall = Neg(pred_sabine_tall)
print(f"{neg_pred_sabine_tall}: {neg_pred_sabine_tall.check(model, f)} \n")

# ... with a double negation
double_neg_pred_sabine_tall = Neg(neg_pred_sabine_tall)
print(f"{double_neg_pred_sabine_tall}: {double_neg_pred_sabine_tall.check(model, f)} \n")

# ... with a variable
neg_pred_y_tall = Neg(pred_y_tall)
print(f"{neg_pred_y_tall}: {neg_pred_y_tall.check(model, VarAssignment({y: 0}))} (with f(y) = 0) \n")

# ... that applies to multiple individuals in the domain
neg_pred_gertrude_human = Neg(pred_gertrude_human)
print(f"{neg_pred_gertrude_human}: {neg_pred_gertrude_human.check(model, f)} \n")

# Binary predicates ...
# ... with a variable
neg_pred_Jordan_like_y = Neg(pred_Jordan_like_y)
print(f"{neg_pred_Jordan_like_y}: {neg_pred_Jordan_like_y.check(model, VarAssignment({y: 2}))} (with f(y) = 2) \n")

# Instructions
*  Same instructions for `Ex`, a sub-class for existential quantification.
*  (You can use the ∃ symbol in `__str__`.)

In [ ]:
# Existencial application
class Ex(Formula):
  # phi: Formula (we assume args list contains var)
  # var: V
  def __init__(self, phi, var):
    check_type(phi, Formula, "phi")
    self._phi = phi
    self._var = var

	# Checks whether the formula is true according to the model `m`
	# m: Model
  def check(self, m, f):
    check_type(m, Model, "m")
    check_type(f, VarAssignment, "f")
    for ind in m.domain: # iterate over the individuals in the domain
      if ( self._phi.check( model, VarAssignment({self._var: ind}) ) ): # if the individual in the domain makes the value of phi true
        return True
    return False # return false if no individual in the domain, that makes phi true, found after the for loop

	# Returns a string representation of the object. Used to print the object in a readable way.
  def __str__(self):
    return f"(∃{self._var}. {self._phi})"

In [ ]:
print(f"The model is:\n {model} \n")

exist_x_wooden = Ex( PredApp(wooden, [x]), x)
print(f"{exist_x_wooden}: {exist_x_wooden.check(model, f)} \n")

exist_y_tall = Ex(pred_y_tall, y)
print(f"{exist_y_tall}: {exist_y_tall.check(model, f)} \n")

exist_Jordan_like_y = Ex(pred_Jordan_like_y, y)
print(f"{exist_Jordan_like_y}: {exist_Jordan_like_y.check(model, f)} \n")

# Instructions
*  Same instructions for `Conj`, a sub-class for conjunction.
*  (You can use the ∧ symbol in `__str__`)

In [ ]:
# Conjunction application
class Conj(Formula):
  # phi, psi: Formula
  def __init__(self, phi, psi):
    check_type(phi, Formula, "phi")
    check_type(psi, Formula, "psi")
    self._phi = phi
    self._psi = psi

	# Checks whether the formula is true according to the model `m` and the variable assignment `f`.
	# m: Model
	# f: VarAssignment
  def check(self, m, f):
    check_type(m, Model, "m")
    check_type(f, VarAssignment, "f")
    return ( (self._phi.check(m, f)) and (self._psi.check(m, f)) ) # return value of phi and value of psi

	# Returns a string representation of the object. Used to print the object in a readable way.
  def __str__(self):
    return f"({self._phi} ∧ {self._psi})"

In [ ]:
print(f"The model is:\n {model} \n")

conj_human_tall = Conj(pred_gertrude_human, PredApp(tall, [Gertrude]))
print(f"{conj_human_tall}: {conj_human_tall.check(model, f)} \n")

conj_formula_negFormula = Conj(pred_Jordan_like_Sabine, Neg(pred_Jordan_like_Sabine))
print(f"{conj_formula_negFormula}: {conj_formula_negFormula.check(model, f)} \n")

print(f"{Neg(conj_human_tall)}: {Neg(conj_human_tall).check(model, f)} \n")

conj_of_conj = Conj(conj_human_tall, pred_sabine_tall)
print(f"{conj_of_conj}: {conj_of_conj.check(model, f)} \n")

exist_y_human_and_Jordan_like_y = Ex( Conj( PredApp(human, [y]), pred_Jordan_like_y ), V("y") )
print(f"{exist_y_human_and_Jordan_like_y}: {exist_y_human_and_Jordan_like_y.check(model, f)} \n")

conj_exist_pred_with_v = Conj(exist_Jordan_like_y, PredApp(human, [y]))
print(f"{conj_exist_pred_with_v}: {conj_exist_pred_with_v.check(model, VarAssignment({y: 3}))} (with f(y) = 3) \n")

# Instructions
*   Test `check_closed` using several complex and diversed closed formulas.

In [ ]:
print(f"the model is {model} \n")

# closed formulas
print(f"check closed with closed formula: {exist_y_human_and_Jordan_like_y}: {exist_y_human_and_Jordan_like_y.check_closed(model)} (same as before)")
print(f"check closed with closed formula: {Neg(conj_human_tall)}: {Neg(conj_human_tall).check_closed(model)} (same as before)")
print(f"check closed with closed formula: {exist_Jordan_like_y}: {exist_Jordan_like_y.check_closed(model)} (same as before) \n ")

# not closed formulas
print(f"check closed when formula is not a closed formula (ex. {pred_y_tall}): (KeyError since no mapping was done for the variable y (empty variable assignment))")
pred_y_tall.check_closed(model)